In [27]:
from gensim import corpora
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from wordcloud import WordCloud, STOPWORDS
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import logging, gensim, bz2
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pyLDAvis.gensim as gensimvis
import pyLDAvis
from sklearn.feature_extraction import DictVectorizer

In [2]:
# start_url = ['http://www.imsdb.com/scripts/']

In [ ]:
# # QT = ['http://www.imsdb.com/scripts/Pulp-Fiction.html',
#       'http://www.imsdb.com/scripts/Django-Unchained.html',
#       'http://www.imsdb.com/scripts/Reservoir-Dogs.html',
#       'http://www.imsdb.com/scripts/Inglourious-Basterds.html', 
#       'http://www.imsdb.com/scripts/True-Romance.html']
#     #create a list of links for writer Quentin Tarantino to create a corpus of his scripts

# PJ = ['http://www.imsdb.com/scripts/Lord-of-the-Rings-Return-of-the-King.html',
#       'http://www.imsdb.com/scripts/Lord-of-the-Rings-Fellowship-of-the-Ring,-The.html',
#       'http://www.imsdb.com/scripts/Lord-of-the-Rings-The-Two-Towers.html',
#       'http://www.imsdb.com/scripts/Heavenly-Creatures.html',
#       'http://www.imsdb.com/scripts/King-Kong.html']
#     #create a list of links for writer Peter Jackson to create a corpusn of his scripts

In [4]:
stops = set(stopwords.words("english"))
# import English stopwords library from nltk to filter useless words from the corpuses 

In [5]:
def Clean_Soup(words, stops):
    # Function to convert a raw script to a string of words
    # The input is a single string (a raw movie script scraped using beautifulsoup) 
    # the output is a single string (a preprocessed movie script to be further vectorized and modeled using gensim)
    
    # Remove HTML
    review_text = bs(words, "lxml").get_text() 
    
    # Remove non-letters    
    review_text = review_text.replace('\\n', ' ')
    review_text = review_text.replace('\n', ' ')
    review_text = review_text.replace('\r', ' ')
    review_text = review_text.replace('\\r', ' ')
    review_text = review_text.replace('\\t', ' ')
    review_text = review_text.replace('\b', ' ')

    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    # Convert to lower case, split into individual words
    # It is importatnt to note that if you do not lowercase all the words, the stopwords might not be removed
    words = letters_only.lower().split() 
    
    # Remove english stop words
    meaningful_words = [w for w in words if not w in stops]

    # Join the words back into one string separated by space, 
    # and return the result.
    return " ".join( meaningful_words )

In [6]:
full_text = str()

In [7]:
def Scraper(r):
    # function iterates through the web links above in cell 3, extracts the text, 
    # isolates the script from that text and then appends it as a string
    full_text = str()
    for i in r:
        html = requests.get(i)
        FullPageofSoup = bs(html.text, 'lxml')
        JustScriptofSoup =  FullPageofSoup.select('.scrtext')
        full_text += JustScriptofSoup[0].get_text()
    return full_text

In [16]:
documents = np.array([Clean_Soup(Scraper(QT), stops), Clean_Soup(Scraper(PJ), stops)])
# create an array to pass into a tokenizer
documents

array([ u'pulp fiction quentin tarantino roger avary pulp fiction quentin tarantino roger avary pulp pulp n soft moist shapeless mass matter magazine book containing lurid subject matter characteristically printed rough unfinished paper american heritage dictionary new college edition int coffee shop morning normal denny spires like coffee shop los angeles morning place jammed healthy number people drinking coffee munching bacon eating eggs two people young man young woman young man slight working class english accent like fellow countryman smokes cigarettes like going style impossible tell young woman old everything contradicts something boy girl sit booth dialogue said rapid pace girl friday fashion young man forget risky doin shit young woman always say thing every time never dangerous young man know always say always right young woman forget day two young man yeah well days forgittin days rememberin begun young woman go like know sound like young man sound like sensible fucking man

In [9]:
texts = map(lambda x: nltk.word_tokenize(x), documents)
# tokenize the documents using ntlk
texts

[[u'pulp',
  u'fiction',
  u'quentin',
  u'tarantino',
  u'roger',
  u'avary',
  u'pulp',
  u'fiction',
  u'quentin',
  u'tarantino',
  u'roger',
  u'avary',
  u'pulp',
  u'pulp',
  u'n',
  u'soft',
  u'moist',
  u'shapeless',
  u'mass',
  u'matter',
  u'magazine',
  u'book',
  u'containing',
  u'lurid',
  u'subject',
  u'matter',
  u'characteristically',
  u'printed',
  u'rough',
  u'unfinished',
  u'paper',
  u'american',
  u'heritage',
  u'dictionary',
  u'new',
  u'college',
  u'edition',
  u'int',
  u'coffee',
  u'shop',
  u'morning',
  u'normal',
  u'denny',
  u'spires',
  u'like',
  u'coffee',
  u'shop',
  u'los',
  u'angeles',
  u'morning',
  u'place',
  u'jammed',
  u'healthy',
  u'number',
  u'people',
  u'drinking',
  u'coffee',
  u'munching',
  u'bacon',
  u'eating',
  u'eggs',
  u'two',
  u'people',
  u'young',
  u'man',
  u'young',
  u'woman',
  u'young',
  u'man',
  u'slight',
  u'working',
  u'class',
  u'english',
  u'accent',
  u'like',
  u'fellow',
  u'countryman',
 

In [10]:
dictionary = corpora.Dictionary(texts)
#creating a dictionary from the tokens
dictionary

In [11]:
corpus = [dictionary.doc2bow(text) for text in texts]
#create a bag of words
#len(corpus) # to check that both documents passed into the corpus
corpus

[[(0, 8),
  (1, 12),
  (2, 12),
  (3, 85),
  (4, 2),
  (5, 9),
  (6, 21),
  (7, 91),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 8),
  (12, 1),
  (13, 5),
  (14, 2),
  (15, 3),
  (16, 4),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 25),
  (21, 2),
  (22, 5),
  (23, 2),
  (24, 14),
  (25, 1),
  (26, 1),
  (27, 3),
  (28, 2),
  (29, 1),
  (30, 1),
  (31, 29),
  (32, 1),
  (33, 2),
  (34, 5),
  (35, 1),
  (36, 36),
  (37, 2),
  (38, 4),
  (39, 2),
  (40, 27),
  (41, 47),
  (42, 10),
  (43, 1),
  (44, 7),
  (45, 1),
  (46, 2),
  (47, 2),
  (48, 2),
  (49, 1),
  (50, 1),
  (51, 12),
  (52, 4),
  (53, 1),
  (54, 1),
  (55, 3),
  (56, 1),
  (57, 12),
  (58, 1),
  (59, 1),
  (60, 22),
  (61, 4),
  (62, 1),
  (63, 3),
  (64, 1),
  (65, 17),
  (66, 3),
  (67, 2),
  (68, 1),
  (69, 1),
  (70, 21),
  (71, 1),
  (72, 20),
  (73, 19),
  (74, 17),
  (75, 1),
  (76, 4),
  (77, 2),
  (78, 2),
  (79, 2),
  (80, 3),
  (81, 18),
  (82, 7),
  (83, 1),
  (84, 2),
  (85, 3),
  (86, 2),
  (87, 6),
  (88, 34),
  (89, 23

In [41]:
# extract 20 LDA topics, using 1 pass and updating once every 1 chunk (10,000 documents)

lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, 
                                      update_every=1, passes=1)

In [42]:
lda.print_topics(5)

[(2,
  u'0.008*frodo + 0.007*mr + 0.005*sam + 0.005*back + 0.005*like + 0.005*django + 0.004*one + 0.004*gandalf + 0.004*go + 0.004*clarence'),
 (6,
  u'0.008*frodo + 0.005*mr + 0.004*looks + 0.004*like + 0.004*back + 0.004*white + 0.004*gandalf + 0.004*one + 0.004*sam + 0.004*ext'),
 (18,
  u'0.006*frodo + 0.005*back + 0.005*mr + 0.005*ext + 0.004*django + 0.004*angle + 0.004*gandalf + 0.004*got + 0.004*white + 0.004*like'),
 (19,
  u'0.007*mr + 0.006*back + 0.005*django + 0.005*schultz + 0.004*frodo + 0.004*white + 0.004*one + 0.004*know + 0.004*day + 0.004*clarence'),
 (15,
  u'0.007*mr + 0.005*frodo + 0.005*django + 0.005*back + 0.005*one + 0.005*know + 0.004*like + 0.004*int + 0.004*white + 0.004*clarence')]

In [43]:
vis_data = gensimvis.prepare(lda, corpus, dictionary, R=15)
pyLDAvis.display(vis_data)